In [23]:
import heapq as pq # Use a priority queue as the data structure for the frontier

In [24]:
def construct_graph(map_file = 'map.txt', distance_file = 'distances.txt'): # assume file are in same relative path a jupyter-notebook
    '''
    Returns the graph and heuristic given a map file and distance file
    
    Parameters:
        map_file(string): map_file as string representation
        distance_file(string): distance_file as string representation
        
    Returns:
            construct_graph(map_file, distance_file): Returns the graph and heuristic given a map file and distance file
    '''
    h = {} # heuristic function representation will be a key value pair, where key is city name and value is the heuristic distance
    f = open(distance_file, 'r')
    for line in f:
        split_index = line.find('-')
        current_city = line[:split_index] # prior to the split index is the city
        distance_to_bucharest = int(line[split_index + 1:]) # everything after split index is the heuristic value of the current city
        h[current_city] = distance_to_bucharest

    f = open(map_file, 'r')
    G = {} # Use an weighted adjacency list representation for the graph 
    for line in f:
        split_index = line.find('-')
        current_city = line[:split_index] # prior to the split index is the city name
        neighbors = line[split_index + 1:] # everything after will be neighbors and weigths
        adj_of_city = []
        for neighbor in neighbors.split(','):  # neighbors are comma-separated
            neighbor = neighbor.rstrip() # remove newline characters
            left_brace_index = neighbor.find('(') # between opening and closing braces will be the distance of the city
            neighbor_name = neighbor[:left_brace_index]
            neighbor_dist = int(neighbor[left_brace_index + 1 : len(neighbor) - 1])
            adj_of_city.append((neighbor_name, neighbor_dist))
        G[current_city] = adj_of_city
    return G, h # return the constructed graph and the heuristic function

In [25]:
def path_to_bucharest(G, h, source):
    '''
    Returns the path and cost to get to Bucharest given a source node. return None if no such path exists
    
    Parameters:
        G (weighted adjacency list): The graph used to represent the state space
        h (dictionary): Dictionary used to retrieve the heuristic value of cities
        source (string): the city name represented as a string
        
    Returns:
        path_to_bucharest(G, h, source): return the path and cost to get to Bucharest given a source node. return None if no such path exists
    '''
    if source not in G: return None # if source is not in the graph no reason to traverse
    g = {source : 0} # distance traveled so far
    f = {source : h[source]} # estimated distance to travel
    frontier = [(f[source], source)]
    parent_pointer = {source : None} # used to find path from destination from source if possible
    while len(frontier) != 0:
        _, current_node = pq.heappop(frontier)
        if current_node == "Bucharest": # A* guarantees that once the target node has been dequeued the solution is optimal
            path = []
            while current_node != None:
                path.append(current_node)
                current_node = parent_pointer[current_node]
            path.reverse()
            return path, g["Bucharest"] # return the path to bucharest and the total distance traveled to get to it

        for neighbor, g_cost in G[current_node]: # traverse nodes adjacent to current node and relax the edge if possible
            if neighbor not in parent_pointer or g[current_node] + g_cost + h[neighbor] < f[neighbor]: # did we find a better f(n), if so update
                g[neighbor] = g[current_node] + g_cost
                f[neighbor] = g[neighbor] + h[neighbor] # evaluation function f(n) = g(n) + h(n) as described in lecture
                parent_pointer[neighbor] = current_node # updating parent pointer for when solution is found we can trace back to source
                pq.heappush(frontier, (f[neighbor], neighbor))

    return None # could not find a path to destination

In [26]:
def pretty_print(path, distance_traveled): # used to conform printing to assignment specifications
    '''
    Pretty prints a path and distance traveled
    
    Parameters:
        path(list): path to bucharest represented as a list
        distance_traveled(float): the distance to travel represented as a float
        
    Returns:
        pretty_print(path, distance_traveled): return None
    '''
    print("From city:", path[0])
    print("To city:", path[len(path) - 1])
    pretty_path = ""
    for city in path[:len(path) - 1]:
        pretty_path += city + " - "
    pretty_path += path[len(path) - 1]
    print("Best route:", pretty_path)
    print("Total distance:", distance_traveled)

In [27]:
G, h = construct_graph()

In [28]:
G # display all the content of the graph

{'Arad': [('Zerind', 75), ('Sibiu', 140), ('Timisoara', 118)],
 'Bucharest': [('Fagaras', 211),
  ('Pitesti', 101),
  ('Giurgiu', 90),
  ('Urziceni', 85)],
 'Craiova': [('RimnicuVilcea', 146), ('Dobreta', 120), ('Pitesti', 138)],
 'Dobreta': [('Mehadia', 75), ('Craiova', 120)],
 'Eforie': [('Hirsova', 86)],
 'Fagaras': [('Sibiu', 99), ('Bucharest', 211)],
 'Giurgiu': [('Bucharest', 90)],
 'Hirsova': [('Urziceni', 98), ('Eforie', 86)],
 'Iasi': [('Vaslui', 92), ('Neamt', 87)],
 'Lugoj': [('Timisoara', 111), ('Mehadia', 70)],
 'Mehadia': [('Lugoj', 70), ('Dobreta', 75)],
 'Neamt': [('Iasi', 87)],
 'Oradea': [('Zerind', 71), ('Sibiu', 151)],
 'Pitesti': [('RimnicuVilcea', 97), ('Bucharest', 101), ('Craiova', 138)],
 'RimnicuVilcea': [('Sibiu', 80), ('Pitesti', 97), ('Craiova', 146)],
 'Sibiu': [('Oradea', 151),
  ('Arad', 140),
  ('Fagaras', 99),
  ('RimnicuVilcea', 80)],
 'Timisoara': [('Arad', 118), ('Lugoj', 111)],
 'Urziceni': [('Bucharest', 85), ('Hirsova', 98), ('Vaslui', 142)],
 'V

In [29]:
h # display all the content of the heuristic function

{'Arad': 366,
 'Bucharest': 0,
 'Craiova': 160,
 'Dobreta': 242,
 'Eforie': 161,
 'Fagaras': 178,
 'Giurgiu': 77,
 'Hirsova': 151,
 'Iasi': 226,
 'Lugoj': 244,
 'Mehadia': 241,
 'Neamt': 234,
 'Oradea': 380,
 'Pitesti': 98,
 'RimnicuVilcea': 193,
 'Sibiu': 253,
 'Timisoara': 329,
 'Urziceni': 80,
 'Vaslui': 199,
 'Zerind': 374}

In [30]:
# go through all cities in the graph and print the best route to Bucharest
for city in G.keys():
    path, cost = path_to_bucharest(G, h, city) or (None, None) # python needs to unpack two elements so we unpack 
    if path is not None:
        pretty_print(path, cost)
        print()

From city: Arad
To city: Bucharest
Best route: Arad - Sibiu - RimnicuVilcea - Pitesti - Bucharest
Total distance: 418

From city: Bucharest
To city: Bucharest
Best route: Bucharest
Total distance: 0

From city: Craiova
To city: Bucharest
Best route: Craiova - Pitesti - Bucharest
Total distance: 239

From city: Dobreta
To city: Bucharest
Best route: Dobreta - Craiova - Pitesti - Bucharest
Total distance: 359

From city: Eforie
To city: Bucharest
Best route: Eforie - Hirsova - Urziceni - Bucharest
Total distance: 269

From city: Fagaras
To city: Bucharest
Best route: Fagaras - Bucharest
Total distance: 211

From city: Giurgiu
To city: Bucharest
Best route: Giurgiu - Bucharest
Total distance: 90

From city: Hirsova
To city: Bucharest
Best route: Hirsova - Urziceni - Bucharest
Total distance: 183

From city: Iasi
To city: Bucharest
Best route: Iasi - Vaslui - Urziceni - Bucharest
Total distance: 319

From city: Lugoj
To city: Bucharest
Best route: Lugoj - Mehadia - Dobreta - Craiova - Pite

In [32]:
# allow user to use program
city_name = input("Enter city name: ")
path_found, cost = path_to_bucharest(G, h, city_name) or (None, None)
if path_found is not None:
    pretty_print(path_found, cost)
else:
    print("No route exists")

Enter city name: Zerind
From city: Zerind
To city: Bucharest
Best route: Zerind - Arad - Sibiu - RimnicuVilcea - Pitesti - Bucharest
Total distance: 493
